# Tweepy scraping

Scrapping twitter handles of popular users from twitter using tweepy. Geotags and hashtags are used to get Kenyan tweets from which user Ids are extracted. The popular users (determined by followers and friends) are added to the dataframe and the less popular ones dropped.

The requests are split into batches in order to have an easy time to pinpoint errors when they arise.

In [8]:
pip install fire

  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=08a5e03be3e79a6039883fa4fe6c53594eec07627995f6516ccec3f767592766
  Stored in directory: c:\users\byuva\appdata\local\pip\cache\wheels\1f\10\06\2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=4629db28bb2dddc8c70bf1ad2d2bb3abe41256e9c58011228a07976d1ef3b125
  Stored in directory: c:\users\byuva\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built fire termcolor
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install preprocessor

Note: you may need to restart the kernel to use updated packages.Building wheels for collected packages: preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4475 sha256=6c2fa7220a2d22dd9ff9f37cfea3cd57b3a3008d53078826a19390a633d37caf
  Stored in directory: c:\users\byuva\appdata\local\pip\cache\wheels\e4\4e\bf\0ecf68aa10ee89d684d90437bd9f89ac19d5dc2921988bb59d
Successfully built preprocessor


In [46]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

import sys
import os
import json
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns

# to view all columns
pd.set_option("display.max.columns", None)

import tweepy
from tweepy.streaming import Stream
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys


import preprocessor as p

In [58]:
consumer_key = ("Consumer Key")
consumer_secret = ("Consumer Secret")
access_token = ("Access Token")
access_token_secret = ("Access Token Secret")


auth =tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = tweepy.API(auth, wait_on_rate_limit=True)
search_query = "Delhi"
places = auth_api.search_tweets(q = search_query , granularity="State")
place_id = places[0].id
places


Unexpected parameter: granularity


[Status(_api=<tweepy.api.API object at 0x0000023C524FF490>, _json={'created_at': 'Sat Nov 20 13:24:33 +0000 2021', 'id': 1462049224028086282, 'id_str': '1462049224028086282', 'text': 'RT @Sydusm: He did not consult any farmer before forcing the bills in Parliament. He did not meet any farmer in a year of protests. He orde…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Sydusm', 'name': 'Shutter Down', 'id': 2876471190, 'id_str': '2876471190', 'indices': [3, 10]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 216032552, 'id_str': '216032552', 'name': 'ASHIS', 'screen_name': 'ashis23', 'location': '', 'description': '', 'url': None,

In [59]:
tweets = tweepy.Cursor(auth_api.search_tweets,q = search_query,
                       geocode="28.7041,77.1025,530km", since_id = '2020-05-01').items(2000)
users = []
followers = []
friends = []
description = []
tweet_count = []

for status in tweets:
    
    name = status.user.screen_name
    desc = status.user.description
    number_of_tweets = status.user.statuses_count
    following = status.user.friends_count
    follower = status.user.followers_count
    users.append(name)
    followers.append(follower)
    friends.append(following)
    description.append(desc)
    tweet_count.append(number_of_tweets) 

    
data = {'screen_name':users, 'followers':followers, 'friends':friends, 'description':description, 'tweets':tweet_count}
users_df = pd.DataFrame(data)
    
#     df['screen_name'] = users
#     df['location'] = location
    
users_df.to_csv('starting users.csv')
users_df.head()

,screen_name,followers,friends,description,tweets
0,KDKedarta,106,32,I’m Nationalist minded & Nature Lover 🇮🇳🌸🌲Anti...,30260
1,PunjabKesariCom,6089,50,National Hindi News Paper,54002
2,aspirationsinst,0,0,#ASPIRATIONS Institute is the premium coaching...,14
3,Dean14094961,436,1041,ji\nMai 21 years ka hu Delhi se\n\nPlz\nMujhe ...,2011
4,studyiq,96750,6,India's Largest Online Education Platform that...,46011


In [64]:
tweets = tweepy.Cursor(auth_api.search_tweets,q = search_query,
                       geocode="28.7041,77.1025,530km", since_id = '2020-06-01').items(2000)
users = []
followers = []
friends = []
description = []
tweet_count = []
for status in tweets:
    name = status.user.screen_name
    desc = status.user.description
    number_of_tweets = status.user.statuses_count
    following = status.user.friends_count
    follower = status.user.followers_count
    users.append(name)
    followers.append(follower)
    friends.append(following)
    description.append(desc)
    tweet_count.append(number_of_tweets) 

data = {'screen_name':users, 'followers':followers, 'friends':friends, 'description':description, 'tweets':tweet_count}
df_2 = pd.DataFrame(data)
    
#     df['screen_name'] = users
#     df['location'] = location
    
df_2.to_csv('df_2 users.csv')
df_2.head()

,screen_name,followers,friends,description,tweets
0,avinashforjesus,3667,4837,BACHELOR PASTOR - Lord Jesus Christ is my God ...,94963
1,avinashforjesus,3667,4837,BACHELOR PASTOR - Lord Jesus Christ is my God ...,94963
2,avinashforjesus,3667,4837,BACHELOR PASTOR - Lord Jesus Christ is my God ...,94963
3,avinashforjesus,3667,4837,BACHELOR PASTOR - Lord Jesus Christ is my God ...,94963
4,avinashforjesus,3667,4837,BACHELOR PASTOR - Lord Jesus Christ is my God ...,94963


In [68]:
tweets = tweepy.Cursor(auth_api.search_tweets,q='Delhi',
                       geocode="28.7041,77.1025,530km", since_id = '2020-07-01').items(2000)
users = []
followers = []
friends = []
description = []
tweet_count = []
for status in tweets:
    name = status.user.screen_name
    desc = status.user.description
    number_of_tweets = status.user.statuses_count
    following = status.user.friends_count
    follower = status.user.followers_count
    users.append(name)
    followers.append(follower)
    friends.append(following)
    description.append(desc)
    tweet_count.append(number_of_tweets) 

data = {'screen_name':users, 'followers':followers, 'friends':friends, 'description':description, 'tweets':tweet_count}
df_3 = pd.DataFrame(data)
    
#     df['screen_name'] = users
#     df['location'] = location
    
df_3.to_csv('df_3 users.csv')
df_3.head()

,screen_name,followers,friends,description,tweets
0,JeetSingh_Rana,1078,2349,"ज्ञान गंगा, जीने की राह, गीता तेरा ज्ञान अमृत\...",15103
1,thetribunechd,133688,303,"Official Twitter handle of The Tribune, North ...",257489
2,itsmesabita,6200,670,Principal Consultant || Executive Search & Str...,19017
3,AAPDelhi,208758,219,Official Twitter Account of #AamAadmiParty - D...,50605
4,esskaydecors,32,64,we provide complete Interior and exterior Deco...,252


In [38]:
df = pd.concat([users_df, df_2, df_3])
df.shape

(2102, 5)

In [39]:
df['popularity'] = df['followers'] - df['friends']
df = df.sort_values(by = 'popularity', ascending = False)
df = df.drop_duplicates(subset ="screen_name") 
df.shape

(1329, 6)

In [40]:
new_df = df[df.popularity >= 50000]
new_df.shape

(73, 6)

In [41]:
new_df

,screen_name,followers,friends,description,tweets,popularity
950,rashtrapatibhvn,15572789,2,This is the official Twitter account of Rashtr...,9676,15572787
1938,timesofindia,14074914,459,News. Views. Analysis. Conversations. India’s ...,712799,14074455
1580,DDNewslive,3348125,94,"Official Twitter account of DD News, the Publi...",205563,3348031
770,ndtvindia,2879937,9,देश के सबसे भरोसेमंद न्यूज़ चैनल 'NDTV इंडिया'...,168307,2879928
1485,airnewsalerts,2869671,109,"Official account of News Services Division, Al...",349522,2869562
...,...,...,...,...,...,...
252,ET_Infra,55984,30,Welcome to ET Infra - India's premier source o...,4271,55954
1200,NewsStateHindi,54893,36,"News State covers breaking news on politics, w...",118876,54857
1789,HindustanTimes,54292,15,One of India's largest media companies. Latest...,442719,54277
1389,khan_zafarul,50513,20,Currently working on English translation of th...,2687,50493


In [48]:
tweets = tweepy.Cursor(auth_api.search_tweets,q = 'Delhi', since_id = '2020-07-01').items(2000)
users = []
followers = []
friends = []
description = []
tweet_count = []
for status in tweets:
    name = status.user.screen_name
    desc = status.user.description
    number_of_tweets = status.user.statuses_count
    following = status.user.friends_count
    follower = status.user.followers_count
    users.append(name)
    followers.append(follower)
    friends.append(following)
    description.append(desc)
    tweet_count.append(number_of_tweets) 

data = {'screen_name':users, 'followers':followers, 'friends':friends, 'description':description, 'tweets':tweet_count}
df_4 = pd.DataFrame(data)
    
#     df['screen_name'] = users
#     df['location'] = location
    
df_4.to_csv('df_4 users.csv')
df_4

Rate limit reached. Sleeping for: 606


,screen_name,followers,friends,description,tweets
0,Mohamme81646244,14,116,All Born equal please stop Discrimination,412
1,kadak_deshbhakt,2088,2167,Sometimes we’re tested not to show our weaknes...,26093
2,MonceAbraham,3020,363,"Thinker, Doer, Pledged Organ Donor | @THEV_Con...",34698
3,iamnkrao,117,903,"॥ ॐ नमः शिवाय ॥ हरे कृष्ण हरे कृष्ण , कृष्ण कृ...",2289
4,kadak_deshbhakt,2088,2167,Sometimes we’re tested not to show our weaknes...,26093
...,...,...,...,...,...
1995,amaresh2212,62,1021,Dreamer/Learner/Keen Observer/Loving the Twitt...,9585
1996,r_ramkrishna,205,457,"friends, waffles, work. or waffles, friends, w...",2770
1997,death_dose19,283,302,Indian Muslim | Hafiz | Hyderabadi 🍆 | Views m...,23728
1998,twittetkabaap,0,0,Oppppp,2
